# Training MLP model on insurance claims data

In [ ]:
from dydx.layers import Linear, Loss
from dydx.model import Model
from dydx.dataset import DataLoader, Dataset

l1 = Linear( random=True, dims=(8, 128))
l2 = Linear( random=True,dims=(128, 256))
l3 = Linear( random=True, dims=(256, 128))
l4 = Linear( random=True,dims=(128, 32))
l5 = Linear( activation=False, random=True, dims=(32,1)
)

		
names = ["l1","l2","l3","l4", "l5"]		
layers =[l1,l2,l3,l4, l5]
ls = dict(zip(names,layers))

model = Model(ls) 
loss = Loss()
EPOCHS= 100

ds = Dataset(testing=True) # takes 160 examples to overfit model too for testing purposes
splits = ['train','val','test']
print(f"dataset sizes for {', '.join(splits)} are {[ds.__len__(split) for split in splits]} respectively.")
dst = ds.train
dsv = ds.val
dss = ds.test
# iterators 
dltrain  = DataLoader(dst,16)
dlval = DataLoader(dsv,16)
dltest = DataLoader(dss,16)


EARLY_STOPPING = None

for epoch in range(EPOCHS):
	print("Training".center(70,"#"))
	for (idx,xy) in enumerate( dltrain()):
		x,y = xy 
		model.zero_grad()	
		y_hat =model(x) 
		
		acc, avg_loss = loss(y,y_hat)
		avg_loss.backward()
        
		
		print(f'Epoch {epoch}:{idx} Loss:{avg_loss.data} train accuracy:{acc*100}%')
			
		lr = 0.0000000005 
		for p in model.parameters():
			#print('p.data:', p.data,  'p.grad:',p.grad)			
			p.data = p.data -  lr * p.grad
        
		# model.zero_grad()

		# # checking if gradients were actually set to zero 
		# for p in model.parameters():
		# 	print('p.grad:',p.grad)

	print("Validation".center(70,"#"))
	for (idx,xy) in enumerate( dlval()):
		x,y = xy 
#			for layer in model.layers:
#				x = layer(x)
		y_hat = model(x)
		acc, avg_loss= loss(y,y_hat)
		print(f'Epoch {epoch}:{idx} Loss:{avg_loss.data} val accuracy:{acc*100}%')	

# train()

dataset seed: 6704240524671637267
dataset seed: 7485732943295468799
dataset sizes for train, val, test are [112, 32, 16] respectively.
dataset sizes for train, val, test are [112, 32, 16] respectively.
###############################Training###############################
Epoch 0:0 Loss:1.2880975663590402 train accuracy:68.75%
Epoch 0:1 Loss:1.2559831934381032 train accuracy:62.5%
Epoch 0:2 Loss:1.7609535161333771 train accuracy:31.25%
Epoch 0:3 Loss:1.5641132072350805 train accuracy:31.25%
Epoch 0:4 Loss:1.340155572469062 train accuracy:43.75%
Epoch 0:5 Loss:1.4538199878477118 train accuracy:31.25%
Epoch 0:6 Loss:1.1879528616516264 train accuracy:43.75%
##############################Validation##############################
Epoch 0:0 Loss:1.2634766381784814 val accuracy:31.25%
Epoch 0:1 Loss:0.8849550960931941 val accuracy:68.75%
###############################Training###############################
Epoch 1:0 Loss:0.8944913737936413 train accuracy:68.75%
Epoch 1:1 Loss:0.93129126667647